In [1]:
#regualr spacy instal lwould not work, had to get from github
#conda install -c conda-forge spacy
#python -m spacy download en
#check with: python -m spacy validate
#restart jupyter notebook

import codecs

import spacy
import pandas as pd
import itertools as it
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

import en_core_web_sm
#nlp = en_core_web_sm.load()
#import en_core_web_md
#nlp = en_core_web_md.load()
nlp = spacy.load('en')

In [2]:
#test
with codecs.open('review_text_lv_rest_subset_30.txt', encoding='utf_8') as f:
    sample_review = list(it.islice(f, 8, 9))[0]
    sample_review = sample_review.replace('\\n', '\n')
        
print(sample_review)

I live very near this place and have been curious to try it for sometime now as it always seems busy.

Well my curiosity has been satisfied and I'm sure I'll never feel the need to visit this place again.

It struck me as a typical trendy place that yuppies feel they need to visit as all the other yuppies do.  
"Oh the food is sooo good!!"
"Really? You've eaten there?"
"No. But all my yuppy friends have and they say it's good so it must be."

I only give it a two because the service and presentation of the food was decent.

We ordered something called oxtail fries which were horrible. I'm sure an actual oxtail would probably have been tastier.

My girlfriend and I each ordered a different burger so we could sample each others. Sorry I can't remember what they were called but we didn't care for either one. The best compliment I could pay them is to say they were at least edible.

So as mentioned the service was decent and the food looked presentable but as far as taste I can't recommend

In [3]:
#helper functions from modern nlp in python
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [5]:
unigram_sentences_filepath = 'unigram_sentences_all_eat_30.txt'

In [6]:
%%time
if 1 == 0:
    with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('review_text_eats_subset_30.txt'):
            f.write(sentence + '\n')

CPU times: user 2d 14h 54min 22s, sys: 1d 16h 18min 7s, total: 4d 7h 12min 29s
Wall time: 15h 9min 21s


In [7]:
%%time
if 1 == 0:
    with codecs.open('unigram_sentences_all_lv_rest_30.txt', 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('review_text_lv_rest_subset_30.txt'):
            f.write(sentence + '\n')

CPU times: user 17h 53min 55s, sys: 12h 6min 47s, total: 1d 6h 43s
Wall time: 4h 29min 35s


In [8]:
unigram_sentences = LineSentence('unigram_sentences_all_lv_rest_30.txt')


In [11]:
for unigram_sentence in it.islice(unigram_sentences, 230, 240):
    print( u' '.join(unigram_sentence))
    print(u'')

the blt be edible

the shrimp po boy be not

-PRON- be not a picky eater but -PRON- be only able to take two bite of the po boy

-PRON- taste like old grease

the shrimp be tasteless

the waiter stop by and ask how the food be

-PRON- tell -PRON- horrible

-PRON- just smile and walk away

-PRON- think -PRON- must have misheard -PRON- lol

other option may be good but -PRON- have only get one shot to make a first impression



In [12]:
bigram_model_filepath = 'bigram_model_lv_rest_30'


In [13]:
%%time

#time consuming
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

CPU times: user 2min 28s, sys: 0 ns, total: 2min 28s
Wall time: 2min 27s


In [14]:
bigram_sentences_filepath = 'bigram_sentences_lv_rest.txt'

In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

/home/rlrognstad/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:486: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
